# Setup the env and the DB

## System & basic packages

```bash
sudo apt install -y build-essential git wget curl lsb-release gnupg
```

## (WSL2 specific)

```bash
echo -e "[boot]\nsystemd=true" | sudo tee /etc/wsl.conf
# wsl --shutdown   # run from PowerShell once
```

## Docker

```bash
curl -fsSL https://get.docker.com | sudo sh
sudo systemctl enable --now docker
sermod -aG docker $USER && newgrp docker
# docker run hello-world                     # should succeed

# Running bash in pg container:
# docker exec -it pg bash
```

### Removing Docker (just in case)

```bash
## sudo apt remove -y docker-ce docker-ce-cli containerd.io \
##   docker-compose-plugin docker-ce-rootless-extras docker-buildx-plugin
## sudo rm -rf /var/lib/docker /var/lib/containerd
```

## PostgreSQL Client and OSM2PGSQL

```bash
sudo apt install -y postgresql-client
# 1. Add the OSM admins PPA which ships fresh builds (1.11 for Jammy)
sudo add-apt-repository ppa:osmadmins/ppa
sudo apt update
sudo apt install -y osm2pgsql
sudo apt install -y pgcli pspg # + add to ~/.bashrc: export PAGER="pspg --csv --no-mouse"
```

## Python

```bash
python -m venv venv && source venv/bin/activate
pip install jupyterlab geopandas h3 duckdb psycopg[binary] sqlalchemy matplotlib tqdm
```

# Data

## Get Cyprus data

```bash
mkdir -p ~/geo/data
wget -O ~/geo/data/cyprus-latest.osm.pbf \
     https://download.geofabrik.de/europe/cyprus-latest.osm.pbf
```

## Run the PG container and mount the folders

```bash
docker run -d --name pg \
  -e POSTGRES_PASSWORD=geo \
  -v $HOME/geo/pgdata:/var/lib/postgresql/data \
  -v $HOME/geo/data:/imports \
  -v $HOME/geo:/geo \
  -p 5432:5432 \
  postgis/postgis:15-3.4 #\
  # -c shared_preload_libraries=pg_stat_statements
```

## Create the DB in PG with all extensions (one-off)

```bash
export PGPASSWORD=geo
psql -h 127.0.0.1 -U postgres -d template_postgis \
  -c "CREATE EXTENSION IF NOT EXISTS hstore;"
dropdb  -h 127.0.0.1 -U postgres --if-exists osm
createdb -h 127.0.0.1 -U postgres -T template_postgis osm
# psql -h 127.0.0.1 -U postgres -d osm -c "\dx"
```

### (Whole bunch of gis-relevant extensions when we need them)
```bash
EXT_LIST="hstore pg_trgm unaccent btree_gist pgrouting postgis_raster postgis_sfcgal"
for ext in $EXT_LIST; do
  PGPASSWORD=geo psql -h 127.0.0.1 -U postgres -d template_postgis \
    -c "CREATE EXTENSION IF NOT EXISTS $ext;"
done
```

## Import Cyprus data

```bash
osm2pgsql --create --slim \
          --cache 1500 \
          --host 127.0.0.1 --port 5432 \
          --user postgres --database osm \
          "$HOME/geo/data/cyprus-latest.osm.pbf"
```